Vamos a comenzar estableciendo una puntuación en función de las derrotas, victorias y empates teniendo en cuenta la ronda en la que estamos, siendo la fase de grupos la menor ponerada:

Fase de Grupos = Puntos * 0.5
RO16 = Puntos * 0.6
RO8 = Puntos * 0.7
RO4 = Puntos * 0.8
SF = Puntos * 0.9
F = Puntos * 1

Dando 3 puntos por victoria, 1 por empate y -1 por derrota

In [13]:
import pandas as pd

def calcular_puntuacion(datos):
    puntuacion_total = {}

    for archivo in datos:
        df = pd.read_csv(archivo)
        for index, row in df.iterrows():
            home_team = row['Home Team']
            home_goals = row['Home Goals']
            away_goals = row['Away Goals']
            away_team = row['Away Team']
            round_number = row['Round Number']

            # Calcular puntos según el resultado
            if home_goals > away_goals:
                puntuacion_local = 3
                puntuacion_visitante = -1
            elif home_goals < away_goals:
                puntuacion_local = -1
                puntuacion_visitante = 3
            else:
                puntuacion_local = 1
                puntuacion_visitante = 1

            # Calcular puntuación ponderada según la ronda
            if round_number == 'Fase de Grupos':
                factor = 0.5
            elif round_number == 'RO16':
                factor = 0.6
            elif round_number == 'RO8':
                factor = 0.7
            elif round_number == 'RO4':
                factor = 0.8
            elif round_number == 'SF':
                factor = 0.9
            else:  # Final
                factor = 1

            # Actualizar puntuación total para el equipo local
            if home_team in puntuacion_total:
                puntuacion_total[home_team] += puntuacion_local * factor
            else:
                puntuacion_total[home_team] = puntuacion_local * factor

            # Actualizar puntuación total para el equipo visitante
            if away_team in puntuacion_total:
                puntuacion_total[away_team] += puntuacion_visitante * factor
            else:
                puntuacion_total[away_team] = puntuacion_visitante * factor

    # Convertir el diccionario de puntuaciones totales a DataFrame
    df_puntuacion = pd.DataFrame(puntuacion_total.items(), columns=['Equipo', 'Puntuacion Total'])

    return df_puntuacion

# Cargar los datos de los archivos CSV
archivos = [
    "Datos_Limpios/UCL2014-2015.csv",
    "Datos_Limpios/UCL2015-2016.csv",
    "Datos_Limpios/UCL2016-2017.csv",
    "Datos_Limpios/UCL2017-2018.csv",
    "Datos_Limpios/UCL2018-2019.csv",
    "Datos_Limpios/UCL2019-2020.csv",
    "Datos_Limpios/UCL2020-2021.csv",
    "Datos_Limpios/UCL2021-2022.csv",
    "Datos_Limpios/UCL2022-2023.csv"
]

# Calcular la puntuación total
puntuacion_total = calcular_puntuacion(archivos)

# Ordenarlo de mayor a menor puntuación

puntuacion_total = puntuacion_total.sort_values(by='Puntuacion Total', ascending=False)

# Mostrar los 5 primeros equipos

print(puntuacion_total.head(5))

# Guardar la puntuación total en un archivo CSV
puntuacion_total.to_csv("Datos_Limpios/Puntaje.csv", index=False)


                 Equipo  Puntuacion Total
0           Real Madrid               195
22         FC Barcelona               184
19        Bayern Munich               180
20  Paris Saint-Germain               150
18      Manchester City               146


Ahora de la tabla Datos/UEFA_Ranking.csv quiero que tomes los valores de los equipos que han salido en la tabla Puntaje.csv

In [15]:
#Ahora de la tabla Datos/UEFA_Ranking.csv quiero que tomes los valores de los equipos que han salido en la tabla Puntaje.csv

# Cargar los datos de los archivos CSV
ranking = pd.read_csv("Datos/UEFA_Ranking.csv")
puntuacion = pd.read_csv("Datos_Limpios/Puntaje.csv")

# Unir solamente la columna Total Points con la tabla Puntaje.csv

puntuacion = puntuacion.merge(ranking[['Unnamed: 2_level_0', 'Total Points']], left_on='Equipo', right_on='Unnamed: 2_level_0')

# Ordenarlo de mayor a menor puntuación

puntuacion = puntuacion.sort_values(by='Puntuacion Total', ascending=False)

# Mostrar los 5 primeros equipos

print(puntuacion.head(5))

# Guardar la puntuación total en un archivo CSV

puntuacion.to_csv("Datos_Limpios/Puntaje.csv", index=False)

KeyError: 'Team'